In [2]:
#import pandas as pd

# --- Input and output paths ---
#file_path = "/Volumes/T7/External Downloads/nasdaq_exteral_data.csv"
#output_path = "nasdaq_titles_dates_symbols.csv"

# --- Columns to read ---
#columns = ['Date', 'Stock_symbol', 'Article_title']

# --- Read and write in chunks ---
#chunksize = 100_000
#first_chunk = True

#reader = pd.read_csv(file_path, usecols=columns, chunksize=chunksize)

#for i, chunk in enumerate(reader):
#    print(f"🔄 Processing chunk {i + 1}")
    
    # Optional: drop rows with missing titles (you can skip this if needed)
#    chunk = chunk.dropna(subset=['Article_title'])

    # Write to CSV
#    chunk.to_csv(
#        output_path,
#        mode='w' if first_chunk else 'a',
#        index=False,
#        header=first_chunk  # write header only for first chunk
#    )
#    first_chunk = False

#print("✅ Done. Saved to:", output_path)


🔄 Processing chunk 1
🔄 Processing chunk 2
🔄 Processing chunk 3
🔄 Processing chunk 4
🔄 Processing chunk 5
🔄 Processing chunk 6
🔄 Processing chunk 7
🔄 Processing chunk 8
🔄 Processing chunk 9
🔄 Processing chunk 10
🔄 Processing chunk 11
🔄 Processing chunk 12
🔄 Processing chunk 13
🔄 Processing chunk 14
🔄 Processing chunk 15
🔄 Processing chunk 16
🔄 Processing chunk 17
🔄 Processing chunk 18
🔄 Processing chunk 19
🔄 Processing chunk 20
🔄 Processing chunk 21
🔄 Processing chunk 22
🔄 Processing chunk 23
🔄 Processing chunk 24
🔄 Processing chunk 25
🔄 Processing chunk 26
🔄 Processing chunk 27
🔄 Processing chunk 28
🔄 Processing chunk 29
🔄 Processing chunk 30
🔄 Processing chunk 31
🔄 Processing chunk 32
🔄 Processing chunk 33
🔄 Processing chunk 34
🔄 Processing chunk 35
🔄 Processing chunk 36
🔄 Processing chunk 37
🔄 Processing chunk 38
🔄 Processing chunk 39
🔄 Processing chunk 40
🔄 Processing chunk 41
🔄 Processing chunk 42
🔄 Processing chunk 43
🔄 Processing chunk 44
🔄 Processing chunk 45
🔄 Processing chunk 

/var/folders/j5/tqvd0k7104s47pv13lpjvlnh0000gn/T/ipykernel_58678/2347666106.py:16: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(reader):


🔄 Processing chunk 66
🔄 Processing chunk 67
🔄 Processing chunk 68
🔄 Processing chunk 69
🔄 Processing chunk 70
🔄 Processing chunk 71
🔄 Processing chunk 72
🔄 Processing chunk 73
🔄 Processing chunk 74
🔄 Processing chunk 75
🔄 Processing chunk 76
🔄 Processing chunk 77
🔄 Processing chunk 78
🔄 Processing chunk 79
🔄 Processing chunk 80
🔄 Processing chunk 81
🔄 Processing chunk 82
🔄 Processing chunk 83
🔄 Processing chunk 84
🔄 Processing chunk 85
🔄 Processing chunk 86
🔄 Processing chunk 87
🔄 Processing chunk 88
🔄 Processing chunk 89
🔄 Processing chunk 90
🔄 Processing chunk 91
🔄 Processing chunk 92
🔄 Processing chunk 93
🔄 Processing chunk 94
🔄 Processing chunk 95
🔄 Processing chunk 96
🔄 Processing chunk 97
🔄 Processing chunk 98
🔄 Processing chunk 99
🔄 Processing chunk 100
🔄 Processing chunk 101
🔄 Processing chunk 102
🔄 Processing chunk 103
🔄 Processing chunk 104
🔄 Processing chunk 105
🔄 Processing chunk 106
🔄 Processing chunk 107
🔄 Processing chunk 108
🔄 Processing chunk 109
🔄 Processing chunk 110

In [7]:
import pandas as pd
df = pd.read_csv("nasdaq_titles_dates_symbols.csv")
df.columns = df.columns.str.lower()
df['date'] = pd.to_datetime(df['date'], utc=True)

/var/folders/j5/tqvd0k7104s47pv13lpjvlnh0000gn/T/ipykernel_64385/2979524305.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("nasdaq_titles_dates_symbols.csv")


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15549298 entries, 0 to 15549297
Data columns (total 3 columns):
 #   Column         Dtype              
---  ------         -----              
 0   date           datetime64[ns, UTC]
 1   article_title  object             
 2   stock_symbol   object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 355.9+ MB


In [11]:
import spacy
import string
import pandas as pd
from tqdm import tqdm
import time

class PreProcessor:
    def __init__(self,
                 lowercase: bool = True,
                 remove_punctuation: bool = True,
                 remove_stopwords: bool = False,
                 lemmatize: bool = False,
                 model: str = 'en_core_web_sm'):

        self.lowercase = lowercase
        self.remove_punctuation = remove_punctuation
        self.remove_stopwords = remove_stopwords
        self.lemmatize = lemmatize

        self.nlp = spacy.load(model, disable=["parser", "tagger"])
        self.stop_words = self.nlp.Defaults.stop_words if remove_stopwords else set()
        self.punctuation_marks = set(string.punctuation)

    def preprocess_doc(self, doc):
        tokens = []
        for token in doc:
            if self.remove_punctuation and token.text in self.punctuation_marks:
                continue
            if self.remove_stopwords and token.text in self.stop_words:
                continue
            if self.lemmatize:
                tokens.append(token.lemma_)
            else:
                tokens.append(token.text)
        return " ".join(tokens)

    def preprocess_batch(self, texts):
        if self.lowercase:
            texts = [text.lower() if isinstance(text, str) else "" for text in texts]
        docs = self.nlp.pipe(texts, batch_size=1000, n_process=1)
        return [self.preprocess_doc(doc) for doc in docs]

    def preprocess_dataframe_in_chunks(self,
                                       df: pd.DataFrame,
                                       text_column: str,
                                       output_csv: str,
                                       chunksize: int = 50000):
        first_chunk = True
        total_rows = len(df)
        for i in tqdm(range(0, total_rows, chunksize), desc="Processing chunks"):
            start_time = time.time()
            chunk = df.iloc[i:i+chunksize].copy()
            texts = chunk[text_column].astype(str).tolist()
            chunk[text_column + "_clean"] = self.preprocess_batch(texts)

            chunk.to_csv(output_csv, mode='w' if first_chunk else 'a',
                         index=False, header=first_chunk)
            first_chunk = False
            print(f"✅ Chunk {i // chunksize + 1} done in {round(time.time() - start_time, 2)}s")


In [12]:
pre = PreProcessor(
    lowercase=True,
    remove_punctuation=True,
    remove_stopwords=True,
    lemmatize=False,
    model="en_core_web_sm"
)

# Assume df_filtered is your in-memory dataframe
pre.preprocess_dataframe_in_chunks(
    df=df,
    text_column="article_title",
    output_csv="/Volumes/T7/External Downloads/nasdaq_titles_fuzzy_rdy.csv"
)


Processing chunks:   0%|                                | 0/311 [00:00<?, ?it/s]/Applications/anaconda3/lib/python3.9/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
Processing chunks:   0%|                     | 1/311 [01:48<9:18:36, 108.12s/it]

✅ Chunk 1 done in 108.12s


/Applications/anaconda3/lib/python3.9/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
Processing chunks:   1%|▏                     | 2/311 [03:12<8:04:14, 94.03s/it]

✅ Chunk 2 done in 84.16s


/Applications/anaconda3/lib/python3.9/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
Processing chunks:   1%|▏                     | 3/311 [04:36<7:40:19, 89.67s/it]

...

✅ Chunk 309 done in 71.15s


/Applications/anaconda3/lib/python3.9/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
Processing chunks: 100%|███████████████████▉| 310/311 [6:37:23<01:11, 71.56s/it]

✅ Chunk 310 done in 70.46s


/Applications/anaconda3/lib/python3.9/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
Processing chunks: 100%|████████████████████| 311/311 [6:38:35<00:00, 76.90s/it]

✅ Chunk 311 done in 71.48s
